## Short TensorBoard Example for Google Colab

Thanks to Tommy Tao for [providing tensorboardcolab](https://github.com/taomanwai/tensorboardcolab) & a [helpful stackoverflow comment](https://stackoverflow.com/questions/48407779/how-to-use-tensorboard-in-colaboratory) by piotr szybicki. 

In [1]:
import numpy as np
import tensorflow as tf
from tensorboardcolab import *

Using TensorFlow backend.


In [2]:
tbc = TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://37cd2f2c.ngrok.io


In [3]:
# task: is input positive or negative?
x = np.random.normal(size=5)
y = [int(i>0) for i in x]
print(np.round(x, 1))
print(y)


[-0.3 -1.1  2.1 -1.2  0.2]
[0, 0, 1, 0, 1]


In [4]:
tf.reset_default_graph()

# get_writer here is analogue to tf.summary.FileWriter()
summary_writer = tbc.get_writer()

x = tf.placeholder(tf.float32, [None, 1], name="x")

with tf.variable_scope("layer_1"):
    w1 = tf.Variable(tf.truncated_normal((1,5)), name="weight")
    b1 = tf.Variable(tf.constant(.1, shape=[5]), name="bias")
    out1 = tf.nn.relu(tf.add(tf.matmul(x,w1), b1), name="out")
    tf.summary.histogram('bias', b1)
    tf.summary.histogram('weight', w1)
    tf.summary.histogram('out', out1)

with tf.variable_scope("layer_2"):
    w2 = tf.Variable(tf.truncated_normal((5,1)), name="weight")
    b2 = tf.Variable(tf.constant(.1), name="bias")
    out = tf.nn.sigmoid(tf.add(tf.matmul(out1,w2), b2), name="out")
    tf.summary.scalar('bias', b2)
    tf.summary.histogram('weight', w2)
    # without squeeze(), out is a vector and scalar() throws an error
    tf.summary.scalar('out', tf.squeeze(out))
    
y = tf.placeholder(tf.float32, [None, 1], name="y")

with tf.variable_scope("loss"):
    loss = tf.losses.mean_squared_error(y, out)
    tf.summary.scalar('loss', loss)   

with tf.variable_scope("training"):
    train_step = tf.train.AdamOptimizer().minimize(loss)

# writing the merged summaries
merged_summary = tf.summary.merge_all()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
# training and creating the summaries

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    for i in range(10000):
        train = np.random.normal(size=1).reshape(1, 1)
        target = np.array([int(i>0) for i in train]).reshape(1,1)

        _, ms = sess.run([train_step, merged_summary], feed_dict={x: train, y: target})
        
        # just like normal TensorBoard, feeding the merged_summary, 
        # the step i and the graph for visualization to the writer
        summary_writer.add_summary(ms, i)
        summary_writer.add_graph(sess.graph)

    

#### Now open the TensorBoard tab and it should look something like this:
![alt text](https://raw.githubusercontent.com/MaxRae/Tensorboard4GoogleColab/master/Media/Bildschirmfoto%202019-02-15%20um%2014.05.17.png)

#### And the graph something like this (after removing training from the main graph):
![alt text](https://raw.githubusercontent.com/MaxRae/Tensorboard4GoogleColab/master/Media/GraphScreenshot.png)

#### And histograms for the vectors like this:
![alt text](https://raw.githubusercontent.com/MaxRae/Tensorboard4GoogleColab/master/Media/HistogramScreenshot.png)